In [1]:
DICT_FILEPATH = './data/dict.opcorpora.xml'

import xml.etree.ElementTree as ET
root = ET.parse(DICT_FILEPATH).getroot()

In [103]:
CORPUS_FILEPATH = './data/annot.opcorpora.no_ambig.xml'
corpus_root = ET.parse(CORPUS_FILEPATH).getroot()

In [245]:
from collections import defaultdict

def canonicalize_tag_name(tag_name):
    if tag_name in ['ADJF', 'ADJS', 'COMP']:
        return 'A'
    elif tag_name in ['NOUN']:
        return 'S'
    elif tag_name in ['VERB', 'INFN', 'PRTF', 'PRTS', 'GRND']:
        return 'V'
    elif tag_name in ['ADVB', 'PRCL', 'INTJ', 'PRED']:
        return 'ADV'
    elif tag_name in ['PREP']:
        return 'PR'
    elif tag_name in ['CONJ']:
        return 'CONJ'
    return tag_name

lemmas = root.findall('lemmata/lemma')
links = root.findall('links/link')
base_lemmas_id = defaultdict()
transformed_to_main = defaultdict(set)
lemmas_by_id = {}
print(len(lemmas))
for lemma in lemmas:
    base_lemmas_id[lemma.get('id')] = lemma.get('id')
    lemmas_by_id[lemma.get('id')] = lemma
for link in links:
    if link.get('type') == '26':
        continue
    base_lemmas_id[link.get('to')] = base_lemmas_id[link.get('from')]
for lemma in lemmas:
    lemma_id = base_lemmas_id[lemma.get('id')]
    #print(lemma_id)
    main_lemma = lemmas_by_id[lemma_id][0]
    main_form = main_lemma.get('t')
    tag = canonicalize_tag_name(main_lemma[0].get('v'))
    for f in lemma[1:]:
        transformed = f.get('t')
        transformed_to_main[transformed].add((main_form, tag))
print(len(transformed_to_main))
test_word = 'стекло'
print(transformed_to_main[test_word])

391243
3058733
{('стекло', 'S'), ('стечь', 'V')}


In [246]:
tokens = corpus_root.findall('./text/paragraphs/paragraph/sentence/tokens/token')
print(len(tokens))
frequencies = defaultdict(lambda: defaultdict(lambda: 1))
for token in tokens:
    transformed = token[0].get('t').lower()
    main = token[0][0][0].get('t')
    tag = canonicalize_tag_name(token[0][0][0][0].get('v'))
    frequencies[transformed][(main, tag)] += 1
print(len(frequencies))
print(frequencies['стекло'])

92983
27433
defaultdict(<function <lambda>.<locals>.<lambda> at 0x7fd4f68560e0>, {('стекло', 'S'): 2})


In [247]:
import numpy as np

def get_best_explanation(transformed):
    if transformed in ["и", "а", "но", "или", "чтобы", "что"]:
        return transformed, "CONJ"

    if transformed in ["по", "на", "у", "за", "для", "при", "через", "до", "среди", "между", "возле"]:
        return transformed, "PR"

    if transformed in ["не", "ни", "затем", "тогда", "итак", "наверно", "бы", "ли", "же", "вот", "только", "уже", "видимо", "потом"]:
        return transformed, "ADV"
    
    if transformed not in transformed_to_main:
        if len(transformed) >= 2 and transformed[-2:] == "ть":
            return transformed, "V"
        if len(transformed) >= 2 and transformed[-2:] == "ся":
            return transformed, "V"
        return transformed, "ADV"
    
    list_of_choices = list(transformed_to_main[transformed])
    freqs = frequencies[transformed]
    probs = np.array([freqs[choice] for choice in list_of_choices])
    probs = probs / sum(probs)
    return list_of_choices[np.random.choice(len(probs), 1, p=probs)[0]]

def proccess_test(test_filepath, output_filepath):
    with open(test_filepath) as test_file, open(output_filepath, 'w') as output_file:
        lines_to_write = proccess_testfile(test_file)
        print(len(lines_to_write))
        output_file.write('\n'.join(lines_to_write))

def proccess_testfile(test_file):
    lines_to_write = []
    for sentence in test_file:
        words = sentence.replace(',', '').replace('.', '').replace('?', '').replace('!', '').replace('\n', '').split(' ')
        preproccessed_words = [word.lower() for word in words]
        words_lemmatization = []
        for word in preproccessed_words:
            main_form, tag = get_best_explanation(word)
            words_lemmatization.append((main_form, tag))
        line_to_write = ' '.join([word + '{' + word_lemmatization[0] + '=' + word_lemmatization[1] + '}' for word, word_lemmatization in zip(words, words_lemmatization)])
        lines_to_write.append(line_to_write)
    return lines_to_write

In [248]:
golden_test = ['Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.\n',
               'Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.']
correct_result = """Стала{стать=V} стабильнее{стабильный=A} экономическая{экономический=A} и{и=CONJ} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR} тени{тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{весь=NI} Гришины{гришин=A} одноклассники{одноклассник=S} уже{уже=ADV} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=NI} был{быть=V} чуть{чуть=ADV} ли{ли=ADV} не{не=ADV} единственным{единственный=A} кого{кто=NI} не{не=ADV} вывозили{вывозить=V} никуда{никуда=NI} дальше{далеко=ADV} Красной{красный=A} Пахры{Пахра=S}"""
result = '\n'.join(proccess_testfile(golden_test))
print(correct_result == result)
print()
print("RESULT")
print(result)
print()
print("EXPECTED")
print(correct_result)

False

RESULT
Стала{стать=V} стабильнее{стабильный=A} экономическая{экономический=A} и{и=CONJ} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR} тени{тенить=V} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{весь=A} Гришины{гришин=A} одноклассники{одноклассник=S} уже{уже=ADV} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=NPRO} был{быть=V} чуть{чуть=ADV} ли{ли=ADV} не{не=ADV} единственным{единственный=A} кого{кто=NPRO} не{не=ADV} вывозили{вывозить=V} никуда{никуда=ADV} дальше{далёкий=A} Красной{красный=A} Пахры{пахра=S}

EXPECTED
Стала{стать=V} стабильнее{стабильный=A} экономическая{экономический=A} и{и=CONJ} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR} тени{тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{весь=NI} Гришины{гришин=A} одноклассники{одноклассник=S} уже{уже=ADV} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=NI} был{быть=

In [249]:
print(transformed_to_main['стала'])
print(transformed_to_main['стабильнее'])

{('стать', 'V')}
{('стабильный', 'A')}


In [252]:
TEST_FILEPATH = './data/test.txt'
OUTPUT_FILEPATH = './data/output.txt'

proccess_test(TEST_FILEPATH, OUTPUT_FILEPATH)

200
